## 환경 설정 & 라이브러리

In [ ]:
from dotenv import load_dotenv
import os

# API KEY 정보 로드
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## 벡터DB 불러오기

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 저장된 벡터DB 로드
vectorstore = FAISS.load_local('../data/db/faiss_counseling', OpenAIEmbeddings())

# 검색기 (Retriever) 설정
retriever = vectorstore.as_retriever()
print("벡터DB 로드 및 검색기 설정 완료.")

## 상담 챗봇 프롬프트 최적화

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 고민을 들어주고 공감하며 상담을 도와주는 AI 챗봇입니다. 
사용자가 고민을 이야기하면 진심 어린 위로와 함께 적절한 질문을 던져 대화를 이어가세요. 
만약 사용자의 고민과 유사한 상담 사례가 있다면, 해당 내용을 기반으로 도움을 주세요. 
그러나 직접적인 해결책을 강요하지 말고, 사용자가 스스로 답을 찾을 수 있도록 도와주세요.

사용자의 감정을 존중하며, 다음 원칙을 따릅니다:
- 고민을 경청하고 따뜻한 말로 공감 표현을 합니다.
- 해결책을 제시하기보다는 열린 질문을 던져 사용자가 답을 찾도록 유도합니다.
- 무조건 긍정하지 말고, 현실적인 조언을 하되 부드러운 표현을 사용합니다.
- 사용자가 감정을 표현하면, 그 감정을 먼저 인정해줍니다.

# 사용자 질문:
{question}

# 상담 사례:
{context}

# 챗봇 응답:
"""
)


## RAG 체인 설정

In [ ]:
# OpenAI LLM 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

# RAG 체인 설정
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## 상담 챗봇 테스트

In [ ]:
# 테스트 질문
question = "요즘 너무 우울해요. 어떻게 해야 할까요?"
answer = rag_chain.invoke(question)
print(answer)

question = "친구랑 싸웠는데 너무 속상해요."
answer = rag_chain.invoke(question)
print(answer)

question = "부모님과의 관계가 힘들어요."
answer = rag_chain.invoke(question)
print(answer)